# Create the embeddings for the query examples

## Prerequisits
- [Ollama](https://ollama.com/search?c=embedding) with the embedding models: `mxbai-embed-large`, `nomic-embed-text`, `all-minilm`
- A pkl file of the classes (check the notebook `Construct_Schema_IDSM_1.ipynb` to generate it)

## Import the required modules

In [ ]:
from tqdm import tqdm
from langchain_ollama import OllamaEmbeddings
from langchain_community.docstore import InMemoryDocstore
import faiss
from langchain_community.vectorstores import FAISS
import logging
from pathlib import Path
import os
import pickle

logging.getLogger("httpx").propagate = False
logging.getLogger("httpx").setLevel("CRITICAL") 

## Prepare the embedding variables

We use the Ollama Embeddings with one of the following models `mxbai-embed-large`, `nomic-embed-text`, `all-minilm`

In [ ]:
embeddings = OllamaEmbeddings(
    model="nomic-embed-text",
)

We use the FAISS vector storage

In [ ]:
vectorstore = FAISS(
    embedding_function=embeddings,
    docstore= InMemoryDocstore(),
    index= faiss.IndexFlatL2(len(embeddings.embed_query("hello world"))),
    index_to_docstore_id={}
)

We initialise the query directory and the saving/loading path. 

Note that all the embeddings are available at [this MyBox URL](https://mybox.inria.fr/d/24d9423c67d64f8284fa/) you can download them to avoid waiting for the embedding process to be done. The password is: `Kc8(-8aE`

## Load the classes from the pickle

In [ ]:
classes_directory = Path(os.getcwd()).parent / 'data' / 'saved_pkls' / 'idsm'

with open(f"{classes_directory}/classes.pkl", 'rb') as handle:
    classes = pickle.load(handle)
    print(len(classes))

In [ ]:
saving_path = Path(os.getcwd()).parent / 'data' / 'faiss_embeddings' / 'idsm' / "v3_4_full_nomic_faiss_index"

Prepare the documents to be injested

In [ ]:
documents = [cls.__str__() for cls in classes[:10]]
len(documents)

## Injest the documents 

In [ ]:
db = None
with tqdm(total=len(documents), desc="Ingesting documents") as pbar:
    for d in documents:
        if db:
            db.add_texts([d])
        else:
            db = FAISS.from_texts([d], embedding=embeddings)
        pbar.update(1)  

## Save the embeddings locally

In [ ]:
db.save_local(saving_path)

## Load the embeeding

In [ ]:
db = FAISS.load_local(saving_path,embeddings=embeddings,allow_dangerous_deserialization=True)
db.index.ntotal

## Example of query selection

In [ ]:
queries = [
    "What protein targets does donepezil (CHEBI_53289) inhibit with an IC50 less than 10 µM?",
    "What protein targets does (CHEBI_124758) inhibit with an PF5 less than 10 µM?",
    "protein targets donepezil (CHEBI_53289) inhibit with IC50",
    "protein targets donepezil (CHEBI_53289) IC50",
    "protein donepezil (CHEBI_53289) IC50",
    "donepezil (CHEBI_53289) IC50",
    "donepezil 53289 IC50"
    ]

query = queries[0]

# Retrieve the most similar text
retrieved_documents = db.similarity_search(query,k=2)

# show the retrieved document's content
for doc in retrieved_documents:
    print(f"{doc.page_content}\n\n-----------------------------------------\n")

## Evaluate the diffrent query forms

This is good to see if a preprocessing task would be benificial

In [ ]:
queries = [
    "What protein targets does donepezil (CHEBI_53289) inhibit with an IC50 less than 10 µM?",
    "protein, targets, donepezil, CHEBI_53289, inhibit, IC50",
    "protein targets donepezil (CHEBI_53289) inhibit with IC50",
    "protein targets donepezil (CHEBI_53289) IC50",
    "protein donepezil (CHEBI_53289) IC50",
    "donepezil (CHEBI_53289) IC50",
    "donepezil 53289 IC50",
    ]

to_find_iris = [
    "http://purl.obolibrary.org/obo/CHEBI_53289",
    "http://www.bioassayontology.org/bao#BAO_0000190",
    "http://www.bioassayontology.org/bao#BAO_0000040",
    "http://purl.obolibrary.org/obo/CHEBI_105741",
    "http://purl.obolibrary.org/obo/CHEBI_109001",
    "http://purl.obolibrary.org/obo/CHEBI_109002" ,
    "http://purl.obolibrary.org/obo/CHEBI_109462" ,
    "http://purl.obolibrary.org/obo/CHEBI_114247" ,
    "http://purl.obolibrary.org/obo/CHEBI_95316"
     ]


best_q = -1
best_score = 0
score = 0

for q in queries:
    score = 0
    # Retrieve the most similar text
    retrieved_documents = db.similarity_search(q,k=20)

    for doc in retrieved_documents:
        for iri in to_find_iris:
            if doc.page_content.find(iri) != -1:
                score = score + 1
        
        if score > best_score:
            best_score = score
            best_q = q

    print(f"{q} => {score}")

print()
print(f"{best_q} => {best_score}")

## Get the context of the selected classes

In [ ]:
import ast
from utils import construct_util


g = construct_util.get_graph_with_prefixes()

for doc in retrieved_documents:
    cls = ast.literal_eval(doc.page_content)
    cls_path = construct_util.format_class_graph_file(cls[0])
    print("Current Classe: "+cls[0])
    cls_context_graph = construct_util.get_context_if_not_found(cls,cls_path)
    g = g + cls_context_graph
    
saving_graph = Path(os.getcwd()).parent / 'tmp' / 'notebook_merging.ttl'

# Save the graph
g.serialize(destination=f"{saving_graph}", format='turtle')


### Embeddings with Chroma Vector Store

The idea would be the same but we need to use the right classes and for saving the embeddings it is **important** to specify the `persist_directory` on the creation.

In [ ]:
from langchain_chroma import Chroma

In [ ]:
embeddings_cn = OllamaEmbeddings(
    model="nomic-embed-text",
)

In [ ]:

saving_path_chroma = Path(os.getcwd()).parent / 'data' / 'chroma_embeddings' / 'idsm' / "v3_4_full_nomic_chroma_index"
dbcn = Chroma(persist_directory=str(saving_path_chroma), embedding_function=embeddings_cn)

In [ ]:

dbcn = None
with tqdm(total=len(documents), desc="Ingesting documents") as pbar:
    for d in documents:
        if dbcn:
            dbcn.add_texts([d])
        else:
            dbcn = Chroma.from_texts(texts=[d], embedding=embeddings_cn,persist_directory=str(saving_path_chroma))
        pbar.update(1)  

And for the search we use:

In [ ]:
dbcn.similarity_search(query="ic50",k=3)